Tóm tắt thông tin mô hình:
- Mô hình sử dụng GXBoost
- Kiểm định whole dataset cross-validation: Để tận dụng toàn bộ tập dataset để train mô hình (vì dataset hiện chỉ giới hạn  4k dòng)


## TD forecast on python

In [1]:
import pyodbc
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import pickle

server = '10.16.157.42'
database = 'RB_DATA'

connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
sql_query = '''
    SELECT *
    FROM USER_DATA.HIENNPD3.FCST_EOP_TD_MAINMODEL 
    WHERE PERIOD LIKE '2023-%' OR PERIOD LIKE  '2024-%'
    ORDER BY PERIOD DESC
'''
cursor.execute(sql_query)  
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)

cursor.close()
conn.close() 


# df = pd.read_csv(r"C:\Users\hiennpd3\OneDrive - VPBank\AA Team\2. TD forecast model\3. Architecture\Package\Model V2\USER_DATA.HIENNPD3.FCST_EOP_TD_MAINMODEL.csv")

In [2]:
df['PERIOD'] = pd.to_datetime(df['PERIOD'])
df_v2 = df.loc[:, ['PERIOD', 'CURRENCY_2', 'TERM_', 'EOP_CM', 'EOP_LM', 'UMASS_MIN', 'UMASS_MAX', 'UMASS_VPB', 'RANKING_UMASS_VPB', 'INTERBANK_SHORT', 'INTERBANK_LONG', 'DISCOUNTRATE', 'REFINANCINGRATE', 'CIE_VOL', 'ADB_VOL']]
df_v2.loc[:, 'YEAR'] = df_v2['PERIOD'].dt.year
df_v2.loc[:, 'MONTH'] = df_v2['PERIOD'].dt.month
df_v2.loc[:, 'NO_ACT_DATE'] = df_v2['PERIOD'].dt.daysinmonth
df_v2

,PERIOD,CURRENCY_2,TERM_,EOP_CM,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,YEAR,MONTH,NO_ACT_DATE
0,2024-04-01,VND,10-11M,NaN,NaN,2.89,4.5,4.1,2,4.68,5.70,3,4.500000,NaN,NaN,2024,4,30
1,2024-04-01,VND,12-18M,NaN,NaN,2.89,4.5,4.1,2,4.68,5.70,3,4.500000,NaN,NaN,2024,4,30
2,2024-04-01,VND,4-5M,NaN,NaN,1.63,2.8,2.3,5,4.35,4.03,3,4.500000,NaN,NaN,2024,4,30
3,2024-04-01,VND,less than 1M,NaN,NaN,1.63,5.0,2.3,5,3.90,3.94,3,4.500000,NaN,NaN,2024,4,30
4,2024-04-01,VND,6-9M,NaN,NaN,2.89,4.5,4.1,2,4.68,5.70,3,4.500000,NaN,NaN,2024,4,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2023-01-01,VND,4-5M,374.504697,282.369630,4.90,6.0,6.0,7,8.08,9.59,4,6.000000,-0.828221,193.075189,2023,1,31
205,2023-01-01,VND,10-11M,359.033516,262.753900,6.00,9.3,8.7,3,10.53,9.61,4,6.000000,-0.942606,161.826927,2023,1,31
206,2023-01-01,NON-VND,From 18M,169.175963,170.980344,6.00,9.3,8.7,3,10.53,9.61,4,6.000000,0.000000,84.687504,2023,1,31
207,2023-01-01,NON-VND,12-18M,682.010259,672.926224,6.00,9.3,8.7,3,10.53,9.61,4,6.000000,-0.029831,346.659790,2023,1,31


In [3]:
from datetime import datetime, timedelta

def get_eop_lm(year, month, term, currency, dataset_name):
    previous_month = datetime(year, month, 1) - timedelta(days=1)
    previous_year = previous_month.year
    previous_month = previous_month.month

    eop_lm = dataset_name.loc[(dataset_name['YEAR'] == previous_year)
                              & (dataset_name['MONTH'] == previous_month)
                              & (dataset_name['TERM_'] == term)
                              & (dataset_name['CURRENCY_2'] == currency), 'EOP_CM'].values

    if len(eop_lm) > 0:
        return eop_lm[0]
    else:
        return np.nan

### 1. Forecast EOP_CM -> Term

In [4]:
df_v3 = df_v2.groupby(['YEAR', 'MONTH', 'NO_ACT_DATE', 'CURRENCY_2', 'TERM_']).agg({'EOP_CM': 'sum'
                                                            , 'EOP_LM': 'sum'                                                                         
                                                            , 'UMASS_MIN': 'max'
                                                            , 'UMASS_MAX': 'max'
                                                            , 'UMASS_VPB': 'max'
                                                            , 'RANKING_UMASS_VPB': 'max'

                                                            , 'INTERBANK_SHORT': 'max'
                                                            , 'INTERBANK_LONG': 'max'
                                                            , 'DISCOUNTRATE': 'max'
                                                            , 'REFINANCINGRATE': 'max'
                                                            
                                                            , 'CIE_VOL': 'sum'
                                                            , 'ADB_VOL': 'sum'}).reset_index()
float_columns = ['EOP_CM','EOP_LM', 'UMASS_MIN', 'UMASS_MAX', 'UMASS_VPB', 'RANKING_UMASS_VPB'
                 , 'INTERBANK_SHORT' , 'INTERBANK_LONG', 'DISCOUNTRATE', 'REFINANCINGRATE' , 'CIE_VOL']

df_v3[float_columns] = df_v3[float_columns].astype(float)


In [5]:
df_v3['EOP_LM_FUNC'] = df_v3.apply(lambda row: row['EOP_LM'] if ((row['YEAR'] == 2023) and (row['MONTH'] == 1)) else get_eop_lm(row['YEAR'],
                                                                                                                                row['MONTH'],
                                                                                                                                row['TERM_'],
                                                                                                                                row['CURRENCY_2'], df_v3),
                                                                                                                                axis=1)
df_v3['CIE_Rate'] = np.where( (df_v3['CIE_VOL'] == 0) | (df_v3['CIE_VOL'] is None) | (df_v3['ADB_VOL'] == 0) |  (df_v3['ADB_VOL'] is None) , 
                             0,
                             (df_v3['CIE_VOL'] / (-df_v3['ADB_VOL'])) * (365/ df_v3['NO_ACT_DATE'] ) * 100 
                             )

In [6]:
mapping_TERM_ = {'less than 1M': 0,
                '1-3M': 1,
                '4-5M': 2,
                '6-9M': 3,
                '10-11M': 4,
                '12-18M': 5,
                'From 18M': 6
                }
mapping_CURRENCY_2 = {'VND': 0, 'NON-VND': 1}

In [7]:
categorical_cols = [ 'CURRENCY_2', 'TERM_']

def encode_engine(df_name, column_name):
    if column_name == 'CURRENCY_2':
        map_rule = mapping_CURRENCY_2
    elif column_name == 'TERM_':
        map_rule = mapping_TERM_
    df_name[column_name] = df_name[column_name].map(map_rule)
    return df_name

for x in categorical_cols:
    df_v4 = encode_engine(df_v3, x)

In [8]:
df_v4[~(df_v4['YEAR'] == 2024)&(df_v4['MONTH'] == 1)]

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_CM,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
0,2023,1,31,1,1,647.556079,564.514299,4.9,6.0,6.0,4.0,8.08,9.59,4.0,6.0,-0.008557,319.663186,564.514299,0.031518
1,2023,1,31,1,5,682.010259,672.926224,6.0,9.3,8.7,3.0,10.53,9.61,4.0,6.0,-0.029831,346.659790,672.926224,0.101320
2,2023,1,31,1,2,2.183222,2.157879,4.9,6.0,6.0,7.0,8.08,9.59,4.0,6.0,0.000000,1.090998,2.157879,0.000000
3,2023,1,31,1,3,952.384486,950.684635,6.0,9.3,8.7,3.0,10.53,9.61,4.0,6.0,-0.031814,483.517485,950.684635,0.077471
4,2023,1,31,1,6,169.175963,170.980344,6.0,9.3,8.7,3.0,10.53,9.61,4.0,6.0,0.000000,84.687504,170.980344,0.000000
5,2023,1,31,1,0,18.883389,184.186660,4.9,6.0,6.0,1.0,5.06,6.04,4.0,6.0,0.000000,9.145337,184.186660,0.000000
6,2023,1,31,0,1,35416.961699,18494.282420,4.9,6.0,6.0,4.0,8.08,9.59,4.0,6.0,-78.800434,16874.328777,18494.282420,5.498361
7,2023,1,31,0,4,359.033516,262.753900,6.0,9.3,8.7,3.0,10.53,9.61,4.0,6.0,-0.942606,161.826927,262.753900,6.858209
8,2023,1,31,0,5,101230.700032,86389.103766,6.0,9.3,8.7,3.0,10.53,9.61,4.0,6.0,-320.612952,47664.370019,86389.103766,7.919876
9,2023,1,31,0,2,374.504697,282.369630,4.9,6.0,6.0,7.0,8.08,9.59,4.0,6.0,-0.828221,193.075189,282.369630,5.050694


In [32]:
current_year = datetime.now().year 
current_month = datetime.now().month
last_month = current_month - 1 if current_month != 1 else 12

df_train = df_v4[~(df_v4['YEAR']==current_year)]
x_train = df_train.drop(['EOP_CM','EOP_LM'], axis=1)
y_train = df_train['EOP_CM']

df_test = df_v4[(df_v4['YEAR']==current_year) & (df_v4['MONTH'] == last_month)]
x_test = df_test.drop(['EOP_CM','EOP_LM'], axis=1)
y_test = df_test['EOP_CM']

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0]
}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=3)
grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_

xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(x_train, y_train)

best_model = grid_search.best_estimator_
# Lưu mô hình vào tệp
filename = 'best_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(best_model, file)

y_train_pred = xgb_model.predict(x_train)
y_test_pre =  xgb_model.predict(x_test)
df_train['FCST_EOP_CM'] = y_train_pred
df_test['FCST_EOP_CM'] = y_test_pre
score_train = xgb_model.score(x_train, y_train)
score_test = xgb_model.score(x_test, y_test)

score_test, score_train


C:\Users\hiennpd3\AppData\Local\Temp\ipykernel_29212\788087423.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['FCST_EOP_CM'] = y_train_pred
C:\Users\hiennpd3\AppData\Local\Temp\ipykernel_29212\788087423.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['FCST_EOP_CM'] = y_test_pre


(0.47024216089597115, 0.9999999994445702)

In [27]:
df_train[df_train['MONTH'] ==1][['YEAR','MONTH','CURRENCY_2','TERM_','EOP_LM_FUNC','EOP_CM','FCST_EOP_CM']]

,YEAR,MONTH,CURRENCY_2,TERM_,EOP_LM_FUNC,EOP_CM,FCST_EOP_CM
0,2023,1,1,1,564.514299,647.556079,633.746521
1,2023,1,1,5,672.926224,682.010259,688.768188
2,2023,1,1,2,2.157879,2.183222,10.213408
3,2023,1,1,3,950.684635,952.384486,942.029358
4,2023,1,1,6,170.980344,169.175963,162.195969
5,2023,1,1,0,184.186660,18.883389,23.634819
6,2023,1,0,1,18494.282420,35416.961699,35418.714844
7,2023,1,0,4,262.753900,359.033516,390.197876
8,2023,1,0,5,86389.103766,101230.700032,101213.695312
9,2023,1,0,2,282.369630,374.504697,375.588470


In [28]:
df_test[df_test['MONTH'] ==3][['YEAR','MONTH','CURRENCY_2','TERM_','EOP_LM_FUNC','EOP_CM','FCST_EOP_CM']]

,YEAR,MONTH,CURRENCY_2,TERM_,EOP_LM_FUNC,EOP_CM,FCST_EOP_CM
182,2024,3,1,1,622.381754,318.012254,313.034912
183,2024,3,1,5,425.605773,216.058772,243.257339
184,2024,3,1,2,5.878347,2.215357,-3.450116
185,2024,3,1,3,359.150542,168.712777,165.161346
186,2024,3,1,6,96.044829,49.460122,19.717737
187,2024,3,1,0,26.357588,17.956073,-2.378502
188,2024,3,0,1,97503.917092,41011.419496,41000.156250
189,2024,3,0,4,759.625728,371.746332,356.247528
190,2024,3,0,5,108307.065373,47208.030495,47190.074219
191,2024,3,0,2,870.575487,397.732804,429.307404


#### Giả định input CIE được truyền vào
Def function để thực hiện predict trong quá trình người nhập dữ liệu

In [12]:
INPUT_CIE_U1M = 0.1
INPUT_CIE_1M3M = 3.1
INPUT_CIE_4M5M = 3.2
INPUT_CIE_6M9M = 3.5
INPUT_CIE_10M11M = 4.5
INPUT_CIE_12M18M = 6.0
INPUT_CIE_OV18M = 7.0

mapping_input = { 0 : INPUT_CIE_U1M,
                  1 : INPUT_CIE_1M3M,
                  2 : INPUT_CIE_4M5M,
                  3 : INPUT_CIE_6M9M,
                  4 : INPUT_CIE_10M11M,
                  5 : INPUT_CIE_12M18M,
                  6 : INPUT_CIE_OV18M
                }

# x_va['CIE_Rate'] =  x_va['TERM_'].map(mapping_input)
# x_va['EOP_CB_FCST'] =  xgb_model.predict(x_va)

from datetime import datetime

mapping_input = { 0 : INPUT_CIE_U1M,
                1 : INPUT_CIE_1M3M,
                2 : INPUT_CIE_4M5M,
                3 : INPUT_CIE_6M9M,
                4 : INPUT_CIE_10M11M,
                5 : INPUT_CIE_12M18M,
                6 : INPUT_CIE_OV18M
                }

current_year = datetime.now().year
current_month = datetime.now().month

x_va['CIE_Rate'] = np.where((x_va['YEAR'] == current_year) & (x_va['MONTH'] == current_month) 
                            , x_va['TERM_'].map(mapping_input)
                            , x_va['CIE_Rate'])
x_va

NameError: name 'x_va' is not defined

In [ ]:
df_train

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_CM,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate,FCST_EOP_CM
0,2023,1,31,1,1,323.778039,282.257150,4.9,6.00,6.0,2.0,8.08,9.59,4.0,6.0,-0.008557,319.663186,282.257150,0.031518,306.679565
1,2023,1,31,1,5,341.005129,336.463112,6.0,9.30,8.7,3.0,10.53,9.61,4.0,6.0,-0.029831,346.659790,336.463112,0.101320,325.159424
2,2023,1,31,1,2,1.091611,1.078939,4.9,6.00,6.0,1.0,8.08,9.59,4.0,6.0,0.000000,1.090998,1.078939,0.000000,59.769451
3,2023,1,31,1,3,476.192243,475.342317,6.0,9.30,8.7,3.0,10.53,9.61,4.0,6.0,-0.031814,483.517485,475.342317,0.077471,446.807953
4,2023,1,31,1,6,84.587982,85.490172,6.0,9.30,8.7,3.0,10.53,9.61,4.0,6.0,0.000000,84.687504,85.490172,0.000000,72.541809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2023,12,31,0,5,65248.915051,63522.272823,3.9,5.50,4.8,11.0,5.08,6.68,3.0,4.5,-489.652166,66091.207769,67182.521857,8.723187,65208.203125
152,2023,12,31,0,2,459.850287,387.146713,2.6,4.35,3.6,5.0,1.33,4.54,3.0,4.5,-1.638594,478.706360,469.056092,4.030262,460.225433
153,2023,12,31,0,3,133466.842511,114831.295974,3.9,5.50,4.8,7.0,5.08,6.68,3.0,4.5,-718.561196,131654.762967,127401.309602,6.426261,132703.375000
154,2023,12,31,0,6,1916.356266,1898.013270,3.9,5.50,4.8,11.0,5.08,6.68,3.0,4.5,-14.027270,1908.714485,1925.689075,8.652933,1936.775146


In [ ]:
x_train

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
0,2023,1,31,1,1,282.257150,4.9,6.00,6.0,2.0,8.08,9.59,4.0,6.0,-0.008557,319.663186,282.257150,0.031518
1,2023,1,31,1,5,336.463112,6.0,9.30,8.7,3.0,10.53,9.61,4.0,6.0,-0.029831,346.659790,336.463112,0.101320
2,2023,1,31,1,2,1.078939,4.9,6.00,6.0,1.0,8.08,9.59,4.0,6.0,0.000000,1.090998,1.078939,0.000000
3,2023,1,31,1,3,475.342317,6.0,9.30,8.7,3.0,10.53,9.61,4.0,6.0,-0.031814,483.517485,475.342317,0.077471
4,2023,1,31,1,6,85.490172,6.0,9.30,8.7,3.0,10.53,9.61,4.0,6.0,0.000000,84.687504,85.490172,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2023,12,31,0,5,63522.272823,3.9,5.50,4.8,11.0,5.08,6.68,3.0,4.5,-489.652166,66091.207769,67182.521857,8.723187
152,2023,12,31,0,2,387.146713,2.6,4.35,3.6,5.0,1.33,4.54,3.0,4.5,-1.638594,478.706360,469.056092,4.030262
153,2023,12,31,0,3,114831.295974,3.9,5.50,4.8,7.0,5.08,6.68,3.0,4.5,-718.561196,131654.762967,127401.309602,6.426261
154,2023,12,31,0,6,1898.013270,3.9,5.50,4.8,11.0,5.08,6.68,3.0,4.5,-14.027270,1908.714485,1925.689075,8.652933


### 2. Forecast NET -> movement_status

### 3. Combine các giá trị ở các năm vào một bảng
- Combine
- Xuất giá trị vào csv
